In [ ]:
!pip install econml

# Tutorial Econml: Generalized Random Forest

In [ ]:
# Load libraries
import pandas as pd
from econml.grf import CausalForest
import matplotlib.pyplot as plt

In [ ]:
# Load the experiment dataset
datos = pd.read_csv("experimento-rfm.csv")
datos

In [ ]:
# Role definition
y = datos['outcome']
T = datos['treat']
X = datos.drop(['id','outcome','treat'], axis=1)
X = pd.get_dummies(X,drop_first=True)


In [ ]:
# Load the new dataset
nuevos = pd.read_csv("experimento-rfm-newdata.csv")

In [ ]:
# Role definition new dataset
X2 = nuevos.drop(['id','Y0','Y1'],axis=1)
X2= pd.get_dummies(X2,drop_first=True)

In [ ]:
# Model: Generalized Random Forest
est = CausalForest(random_state=1234)
est.fit(X, T, y)
efecto = est.predict(X2).ravel()
# Map feature importances to feature names
feature_names = X2.columns.tolist()  # Get the names of the variables from the X matrix
feature_importances = est.feature_importances_ # Get feature importances from output



In [ ]:
# Map feature importances to feature names
feature_names = X2.columns.tolist()  # Get the names of the variables from the X matrix
feature_importances = est.feature_importances_ # Get feature importances from output

# Convert to a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

feature_importance_df

In [ ]:
# Sort and plot
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()


In [ ]:
# Create dataframe with id and estimate of CATE, then export them to a csv file
pronosticos = pd.DataFrame({'id': nuevos['id'],'tau': efecto})
pronosticos.to_csv('pronosticos.csv',index=False)
pronosticos